In [ ]:
import numpy as np
import pandas as pd


# =======================================================
# --- Functions ---
# =======================================================
def simulate_one_run_of_particle_system(
    gamma, beta, v, x0, T, K, N
):
    h = T / K  # Time step size
    X = np.full(N, x0) # Particle values for this single run

    for _ in range(K): # Time stepping
        dW = np.sqrt(h) * np.random.randn(N)
        mean_X = np.mean(X)
        drift_term = (gamma * X + beta * mean_X) * h
        diffusion_term = v * dW
        X = X + drift_term + diffusion_term
    
    return np.mean(X)  # Return the mean of all particles at time T


def estimate_moment_and_get_terminal_values(
    gamma, beta, v, x0, T, K, N, num_runs
):
    terminal_values_X1 = np.zeros(num_runs)

    for run_idx in range(num_runs):
        
        terminal_values_X1[run_idx] = simulate_one_run_of_particle_system(
            gamma, beta, v, x0, T, K, N
        )
    return terminal_values_X1


def calculate_first_moment(gamma, beta, x0, T, K):
    h = T / K
    first_moment = (1 + (gamma + beta) * h)**K * x0
    return first_moment

def calculate_precision(empirical_mean, num_runs):
    variance = np.var(empirical_mean, ddof=1) # ddof=1 for unbiased estimatior of variance
    precision = 1.96 * np.sqrt(variance / num_runs)
    return precision


# =======================================================
# --- Parameters ---
# =======================================================
# gamma + beta = 0.3, does not matter which one is which

# gamma = 1/2 # paper wrote
# beta = 4/5 # paper wrote
gamma = 0.1 # paper may used
beta = 0.2 # paper may used
v_squared = 0.5
v = np.sqrt(v_squared)
x0 = 1
T = 1
K = 50
num_runs = 5 * 10**3 # for quick testing, paper used 5 * 10**6
N_values = [20, 40, 80, 160, 320]


# =======================================================
# --- Main Simulation Loop ---
# =======================================================
first_moment = calculate_first_moment(
    gamma, beta, x0, T, K
)
print(f"Closed-form discretized value for E[X_T^(1,N,h)]: {first_moment:.5f}\n")

table_data = []

for N in N_values:
    print(f"Running for N = {N} particles...")

    # Get all terminal values from num_runs simulations
    terminal_values_X1 = estimate_moment_and_get_terminal_values(
        gamma, beta, v, x0, T, K, N, num_runs
    )

    estimated_first_moment = np.mean(terminal_values_X1)
    difference = estimated_first_moment - first_moment
    precision = calculate_precision(terminal_values_X1, num_runs)

    table_data.append({
        "Nb. particles": N,
        "Estimated first moment": estimated_first_moment,
        "Difference": difference,
        "Precision": precision
    })
    # print(f"Finished N = {N}. Estimated moment: {estimated_moment:.5f}, Difference: {difference:.5f}, Precision: {precision:.5f}\n")


# =======================================================
# --- Table Generation ---
# =======================================================
df_table1 = pd.DataFrame(table_data)

print(f"Closed-form discretized value (reference for 'Difference'): {first_moment:.5f}")
desired_columns = ["Nb. particles", "Estimated first moment", "Difference", "Precision"]
df_print = df_table1[desired_columns].copy()
df_print["Estimated first moment"] = df_print["Estimated first moment"].map('{:.5f}'.format)
df_print["Difference"] = df_print["Difference"].map('{:.5f}'.format) # Adjust format if very small values
df_print["Precision"] = df_print["Precision"].map('{:.5f}'.format)
print(df_print.to_string(index=False))


# =======================================================
# --- Scaled Precision Table ---
# =======================================================
print("\n\n--- Estimated Precision if num_runs were 5 * 10^6 ---")
df_scaled_precision = df_table1[["Nb. particles", "Precision"]].copy() # Use original float precision

target_num_runs = 5 * 10**6
scaling_factor = np.sqrt(num_runs / target_num_runs)

df_scaled_precision["Scaled Precision (for 5e6 runs)"] = df_scaled_precision["Precision"] * scaling_factor
df_scaled_precision_print = df_scaled_precision[["Nb. particles", "Scaled Precision (for 5e6 runs)"]].copy()
df_scaled_precision_print["Scaled Precision (for 5e6 runs)"] = \
    df_scaled_precision_print["Scaled Precision (for 5e6 runs)"].map('{:.5f}'.format)
print(df_scaled_precision_print.to_string(index=False))

Closed-form discretized value for E[X_T^(1,N,h)]: 1.34865

Running for N = 20 particles...
Running for N = 40 particles...
Running for N = 80 particles...
Running for N = 160 particles...
Running for N = 320 particles...
Closed-form discretized value (reference for 'Difference'): 1.34865
 Nb. particles Estimated first moment Difference Precision
            20                1.34919    0.00054   0.00508
            40                1.34731   -0.00134   0.00358
            80                1.34752   -0.00113   0.00255
           160                1.34879    0.00014   0.00179
           320                1.34832   -0.00033   0.00127


--- Estimated Precision if num_runs were 5 * 10^6 ---
 Nb. particles Scaled Precision (for 5e6 runs)
            20                         0.00016
            40                         0.00011
            80                         0.00008
           160                         0.00006
           320                         0.00004
